In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from scipy.ndimage import shift
import math

# Data preparation

In [2]:
NM_data = pd.read_csv('../data/LMKS_NM_1981-2023_hour.csv')
data = pd.read_csv('../data/omni_full_1964-2022.csv')
data

,Unnamed: 0,time1,Rot$,IMF,PLS,IMF_PTS,PLS_PTS,ABS_B,F,THETA_AV,...,F10_INDEX+48,BZ_GSE+1,BZ_GSE+2,BZ_GSE+3,BZ_GSE+4,BZ_GSE+6,BZ_GSE+8,BZ_GSE+12,BZ_GSE+24,BZ_GSE+48
0,0,1963-01-01 01:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1963-01-01 02:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1963-01-01 03:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1963-01-01 04:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1963-01-01 05:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520438,520438,2022-05-13 12:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520439,520439,2022-05-13 13:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520440,520440,2022-05-13 14:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520441,520441,2022-05-13 15:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
NM_data.index = NM_data['Unnamed: 0']
NM_data.drop("Unnamed: 0", axis=1, inplace=True)
NM_data

,H_COR
Unnamed: 0,
1981-12-01 00:00:00,87.2680
1981-12-01 01:00:00,86.9240
1981-12-01 02:00:00,86.8670
1981-12-01 03:00:00,86.4080
1981-12-01 04:00:00,86.5230
...,...
2023-07-10 19:00:00,92.4196
2023-07-10 20:00:00,92.4196
2023-07-10 21:00:00,92.4196


In [4]:
NM_data.dropna(inplace=True)
NM_data.isna().sum()

H_COR    0
dtype: int64

In [5]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data.index = data['time1']
data.drop('time1', axis=1, inplace=True)
data

,Rot$,IMF,PLS,IMF_PTS,PLS_PTS,ABS_B,F,THETA_AV,PHI_AV,BX_GSE,...,F10_INDEX+48,BZ_GSE+1,BZ_GSE+2,BZ_GSE+3,BZ_GSE+4,BZ_GSE+6,BZ_GSE+8,BZ_GSE+12,BZ_GSE+24,BZ_GSE+48
time1,,,,,,,,,,,,,,,,,,,,,
1963-01-01 01:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-01-01 02:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-01-01 03:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-01-01 04:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-01-01 05:00:00,1771.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 12:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-13 13:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-13 14:00:00,9999.0,99.0,99.0,999.0,999.0,NaN,NaN,NaN,NaN,NaN,...,135.600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_relevant = data[['BZ_GSE', 'V', 'N', 'DST', 'SIGMA$Bz']].loc["1995-01-01 00:00:00":]
data_relevant.isna().sum()

BZ_GSE       977
V            903
N           5586
DST            0
SIGMA$Bz     977
dtype: int64

In [7]:
data_relevant.index = pd.to_datetime(data_relevant.index)
data_relevant_interpolated = data_relevant.interpolate(method='time')
data_relevant_interpolated.isna().sum()

BZ_GSE      0
V           0
N           0
DST         0
SIGMA$Bz    0
dtype: int64

## Intersection of omni data and NM data

In [8]:
data_relevant_interpolated.index = pd.to_datetime(data_relevant_interpolated.index)
NM_data.index = pd.to_datetime(NM_data.index)

common_index = data_relevant_interpolated.index.intersection(NM_data.index)

data_relevant_interpolated_common = data_relevant_interpolated.loc[common_index]
nm_data_common = NM_data.loc[common_index]

data_relevant_interpolated_common['NM'] = nm_data_common.iloc[:, 0]
data_relevant_interpolated = data_relevant_interpolated_common

data_relevant_interpolated.isna().sum()

BZ_GSE      0
V           0
N           0
DST         0
SIGMA$Bz    0
NM          0
dtype: int64

## Data normalization

In [9]:
min_val = data_relevant_interpolated.min()
max_val = data_relevant_interpolated.max()

# Normalize between -1 and 1
data_relevant_interpolated_normalized = 2 * (data_relevant_interpolated - min_val) / (max_val - min_val) - 1
data_relevant_interpolated_normalized

,BZ_GSE,V,N,DST,SIGMA$Bz,NM
1995-01-01 00:00:00,0.131579,-0.818939,-0.763676,0.683367,-0.976134,0.412806
1995-01-01 00:00:00,0.131579,-0.818939,-0.763676,0.683367,-0.976134,0.412806
1995-01-01 01:00:00,0.149123,-0.818939,-0.727206,0.703407,-0.961814,0.405224
1995-01-01 02:00:00,0.168860,-0.808533,-0.719912,0.715431,-0.914081,0.403689
1995-01-01 03:00:00,0.184211,-0.814776,-0.762217,0.711423,-0.961814,0.399131
...,...,...,...,...,...,...
2022-05-13 12:00:00,0.184211,-0.829344,-0.905179,0.699399,-0.904535,0.714550
2022-05-13 13:00:00,0.184211,-0.829344,-0.905179,0.707415,-0.904535,0.715932
2022-05-13 14:00:00,0.184211,-0.829344,-0.905179,0.711423,-0.904535,0.717940
2022-05-13 15:00:00,0.184211,-0.829344,-0.905179,0.711423,-0.904535,0.707895


# Regression model

In [10]:
data_relevant_interpolated_normalized

,BZ_GSE,V,N,DST,SIGMA$Bz,NM
1995-01-01 00:00:00,0.131579,-0.818939,-0.763676,0.683367,-0.976134,0.412806
1995-01-01 00:00:00,0.131579,-0.818939,-0.763676,0.683367,-0.976134,0.412806
1995-01-01 01:00:00,0.149123,-0.818939,-0.727206,0.703407,-0.961814,0.405224
1995-01-01 02:00:00,0.168860,-0.808533,-0.719912,0.715431,-0.914081,0.403689
1995-01-01 03:00:00,0.184211,-0.814776,-0.762217,0.711423,-0.961814,0.399131
...,...,...,...,...,...,...
2022-05-13 12:00:00,0.184211,-0.829344,-0.905179,0.699399,-0.904535,0.714550
2022-05-13 13:00:00,0.184211,-0.829344,-0.905179,0.707415,-0.904535,0.715932
2022-05-13 14:00:00,0.184211,-0.829344,-0.905179,0.711423,-0.904535,0.717940
2022-05-13 15:00:00,0.184211,-0.829344,-0.905179,0.711423,-0.904535,0.707895


In [11]:
DST_shift = data_relevant_interpolated_normalized['DST'].shift(1)
DST_shift.fillna(value=0,inplace=True)
DST_shift

1995-01-01 00:00:00    0.000000
1995-01-01 00:00:00    0.683367
1995-01-01 01:00:00    0.683367
1995-01-01 02:00:00    0.703407
1995-01-01 03:00:00    0.715431
                         ...   
2022-05-13 12:00:00    0.699399
2022-05-13 13:00:00    0.699399
2022-05-13 14:00:00    0.707415
2022-05-13 15:00:00    0.711423
2022-05-13 16:00:00    0.711423
Name: DST, Length: 239877, dtype: float64

In [12]:
N = data_relevant_interpolated_normalized['N'].to_numpy()
V = data_relevant_interpolated_normalized['V'].to_numpy()
Bz= data_relevant_interpolated_normalized['BZ_GSE'].to_numpy()
sigma_bz = data_relevant_interpolated_normalized['SIGMA$Bz'].to_numpy()
nm = data_relevant_interpolated_normalized['NM'].to_numpy()

Definicia persistancneho modela ako linearana kombinacia zvysnych parametrov modelu FFNN.
$$
DST_{t+1} = DST_t + \sum_{i = 1}^{|N|} = \omega_i(\psi_i - \psi_i^{ref})
$$
To iste v podobe matic a vektorov.
$$
\vec{y} = \vec{\beta} + \mathbf{X}\vec{\omega}
$$

In [13]:
y = DST_shift.to_numpy()
b = data_relevant_interpolated_normalized['DST'].to_numpy()
b.shape, y.shape

((239877,), (239877,))

## Vypocet matice $X$

In [14]:
X = []

In [15]:
N_mean = N.mean()
V_mean = V.mean()
Bz_mean = Bz.mean()
sigma_bz_mean = sigma_bz.mean()
nm_mean = nm.mean()
N_mean, V_mean, Bz_mean, sigma_bz_mean, nm_mean

(np.float64(-0.9092295346299987),
 np.float64(-0.5875355186766423),
 np.float64(0.1771662958918724),
 np.float64(-0.9377969344570631),
 np.float64(0.4876614210538083))

In [16]:
params = [N, V, Bz, sigma_bz, nm]

In [17]:
X = np.array([
    N - N_mean, V - V_mean, Bz - Bz_mean, sigma_bz - sigma_bz_mean, nm - nm_mean
])
X.shape, X.T.shape

((5, 239877), (239877, 5))

In [18]:
X = X.T
X.shape

(239877, 5)

## Vypocet vah
$$
\vec{\omega} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T(\vec{y} - \vec{\beta})
$$

In [19]:
w = (np.linalg.inv(X.T @ X) @ X.T) @ (y - b)

In [20]:
w

array([-0.00207664, -0.00092434, -0.08359456,  0.03263757,  0.00126522])

## Finalny model

In [21]:
persistance = b + np.sum(X*w, axis=1)

In [22]:
persistance

array([0.6857433 , 0.6857433 , 0.70469884, ..., 0.71242604, 0.71241333,
       0.7083976 ], shape=(239877,))

In [23]:
%matplotlib qt
plt.plot(y, label='DST_t+1')
plt.plot(persistance, label='prediction')
plt.legend()

In [24]:
mean_squared_error(y, persistance)

0.00026118515268803424

# Shapely values
Nech $\psi$ je nejaky parameter, $P$ je mnozina parametrov a $f : P \rightarrow DST_{t+1}$ je model a $x_0$ je jedna hodnota DST. Potom shapely hodnota parametru $\psi$ v bode $x_0$ je
$$
\varphi_{\psi}(x_0) = \frac{1}{|P|!} \sum_{S \subseteq P \setminus \{\psi\}} |S|!\cdot(|P| - |S| - 1) \cdot (f(S \cup \{\psi\}) - f(S))
$$
Vypocet si bude vyzadovat postavit niekolko krokov.
- Napjprv je potrebne spravit **mnozinu podmnozin** mnoziny $P$
- Vytvorit **funkciu** ktora **vytvori model s novym poctom parametrov** na vstupe, kedze $f(S)$ bude vzdy predikcia modelu s inym poctom parametrov.
- Skombinovat

## Mnozina podmnozin $P$
Na $S \subseteq P \setminus \{\psi\}$ sa mozeme pozerat trochu inak. V podstate to hovori ze $S$ je kazda podmnozina mnoziny $P$. Preto mozeme zapisat $S$ ako $S \subseteq \mathcal{P(\mathbf{P})}$, cize $S$ je kazda mnozina potencnej mnozinu $P$.

In [25]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [26]:
AllParams = [N, V, Bz, sigma_bz, nm, data_relevant_interpolated_normalized['DST'].to_numpy()]

In [27]:
def power_set_minus(P, target):
    new_P = []
    for i in P:
        if i.all() == target.all(): continue
        else: new_P.append(i)
    power_P = list(powerset(new_P))

    print(len(power_P))
    return power_P

## Dynamicky persistencny model

In [79]:
def f(P, y, b):
    '''
        P - set of parameters ([N, V, Bz,...]) (2D matrix)
        y - DST_t+1 (array)
        b - DST_t   (array)
    '''
    if not P: return []
    X = np.array([
        p - p.mean() for p in P 
    ])
    X = X.T

    w = (np.linalg.inv(X.T @ X) @ X.T) @ (y - b)

    return w

def pp(X, b, w):
    '''
        X - shape [p1 - p1_mean, p2 - p2_mean, ...]
        b - shape [DST1, DST2, DST3]
        w - weights for parameters p_1 - p_n
    '''
    X = X.T
    if len(X.shape) <= 1: return b + X*w
    else: return b + np.sum(X*w, axis=1)
    

In [58]:
f([], y, b)

[]

In [71]:
pp(
    np.array([N[:10] - N.mean(), V[:10] - V.mean(), Bz[:10] - Bz.mean(), sigma_bz[:10] - sigma_bz.mean(), nm[:10] - nm.mean()]),
    b[:10],
    f([N, V, Bz, sigma_bz, nm], y, b)
)

array([0.6857433 , 0.6857433 , 0.70469884, 0.71660417, 0.70984288,
       0.70092389, 0.70507789, 0.70693801, 0.70247071, 0.69478835])

## Shapely values calculation

In [85]:
def shapely(P, target, point):
    '''
    P - set of parameters
    target - parameter for which we are calculating shapely value
    point - int value in range of the size of the parameters from P
    '''
    # Creating set of parameters without target parameter
    mask = ~np.all(P == target, axis=1)
    indexes = [i for i in range(len(mask)) if mask[i] == True]
    P_wo_target = [P[i] for i in indexes]

    # power set of set w/o target
    p_set = list(powerset(P_wo_target))[1:]

    # main loop 'sum' of the shapely formula
    summ = 0
    for sub_set in p_set:

        # finding the position of DST in every subset
        # because we need to extract DST out of the whole array as it is treated differently in comparison to other parameters
        mask_dst = ~np.all(sub_set == data_relevant_interpolated_normalized['DST'].to_numpy(), axis=1)
        indexes_dst = [i for i in range(len(mask_dst)) if mask_dst[i] == False]

        # Creating X matrix and b vector for linear model
        # based on the existance of DST in subset
        X = []
        b = []
        if not indexes_dst: # indexes_dst is empty, no dst in that subset
            X = [p[point] - p.mean() for p in sub_set]
            b = np.zeros(len(y))
        else: # DST is in the subset, we need to extract it
            X = [p[point] - p.mean() for p in sub_set[:indexes_dst[0]]]
            b = sub_set[indexes_dst[0]]
            sub_set = list(sub_set)
            del sub_set[indexes_dst[0]]

        # contribution value f(S U {i}) - f(S)
        prediction          = pp(np.array(X), b[point], f(sub_set, y, b))
        X.append(target[point] - target.mean())
        prediction_w_target = pp(np.array(X), b[point], f(sub_set, y, b))

        contribution = prediction_w_target - prediction

        # # multiplication factor |S|! * (|P| - |S| - 1)!
        # mult_factor = math.factorial(len(sub_set)) * math.factorial((len(P) - len(S) - 1))

        # summ += mult_factor * contribution
        

In [86]:
shapely(
    [N, V, Bz, sigma_bz, nm, data_relevant_interpolated_normalized["DST"].to_numpy()],
    N,
    25
)

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [33]:
y - np.zeros(len(y)), y

(array([0.        , 0.68336673, 0.68336673, ..., 0.70741483, 0.71142285,
        0.71142285], shape=(239877,)),
 array([0.        , 0.68336673, 0.68336673, ..., 0.70741483, 0.71142285,
        0.71142285], shape=(239877,)))

In [49]:
idk = list(list(powerset(["V", "Bz", "sBz", "nm", "DST"]))[20])

In [50]:
idk

['V', 'sBz', 'DST']

In [51]:
del idk[2]
idk

['V', 'sBz']